In [1]:
import os
import fire

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (
    DataLoader, 
    RandomSampler, 
    Dataset, 
    SequentialSampler
)

import torch.nn.functional as F
from tqdm import tqdm, trange

from transformers import (
    RobertaConfig, 
    RobertaTokenizer, 
    RobertaForSequenceClassification
)
from transformers import set_seed

from datasets import load_dataset

# Mixup Roberta

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union

from transformers.models.roberta.modeling_roberta import (
    RobertaPreTrainedModel,
    RobertaLayer,
    RobertaEmbeddings,
    RobertaPooler,
    RobertaClassificationHead
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    SequenceClassifierOutput
)

In [3]:
FLAG = True
LAMBDA = 0.5

In [4]:
class RobertaMixerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([RobertaLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )

In [5]:
class RobertaMixerModel(RobertaPreTrainedModel):
    """
    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in *Attention is
    all you need*_ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz
    Kaiser and Illia Polosukhin.
    To behave as an decoder the model needs to be initialized with the `is_decoder` argument of the configuration set
    to `True`. To be used in a Seq2Seq model, the model needs to initialized with both `is_decoder` argument and
    `add_cross_attention` set to `True`; an `encoder_hidden_states` is then expected as an input to the forward pass.
    .. _*Attention is all you need*: https://arxiv.org/abs/1706.03762
    """

    _keys_to_ignore_on_load_missing = [r"position_ids"]

    # Copied from transformers.models.bert.modeling_bert.BertModel.__init__ with Bert->Roberta
    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = RobertaEmbeddings(config)
        self.encoder = RobertaMixerEncoder(config)

        self.pooler = RobertaPooler(config) if add_pooling_layer else None

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)


            
    # Copied from transformers.models.bert.modeling_bert.BertModel.forward
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


In [6]:
class RobertaMixerForSequenceClassification(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaMixerModel(config, add_pooling_layer=False)
        
        if FLAG:
            self.mixup_dense = nn.Linear(config.hidden_size, config.hidden_size)
            self.mixup_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
            self.mixup_dropout = nn.Dropout(config.hidden_dropout_prob)

        self.classifier = RobertaClassificationHead(config)

        # Initialize weights and apply final processing
        self.post_init()

    
    
    def forward(
        self,
        input_ids_1: Optional[torch.LongTensor] = None,
        attention_mask_1: Optional[torch.FloatTensor] = None,
        token_type_ids_1: Optional[torch.LongTensor] = None,
        input_ids_2: Optional[torch.LongTensor] = None,
        attention_mask_2: Optional[torch.FloatTensor] = None,
        token_type_ids_2: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels_1: Optional[torch.LongTensor] = None,
        labels_2: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs_1 = self.roberta(
            input_ids_1,
            attention_mask=attention_mask_1,
            token_type_ids=token_type_ids_1,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output_1 = outputs_1[0]
        
        # Mixup train
        if (input_ids_2 is not None) and (attention_mask_2 is not None) and (labels_2 is not None):
            
            outputs_2 = self.roberta(
                input_ids_2,
                attention_mask=attention_mask_2,
                token_type_ids=token_type_ids_2,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            sequence_output_2 = outputs_2[0]

            sequence_output = (LAMBDA * sequence_output_1) + ((1 - LAMBDA) * sequence_output_2)

            if FLAG:
                sequence_output = self.mixup_dense(sequence_output)
                sequence_output = self.mixup_layernorm(sequence_output)
                sequence_output = self.mixup_dropout(sequence_output)

            logits = self.classifier(sequence_output)

            loss = None
            loss_fct = nn.CrossEntropyLoss()
            loss = (LAMBDA * loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))) + ((1 - LAMBDA) * loss_fct(logits.view(-1, self.num_labels), labels_2.view(-1)))

        # Mixup eval
        else:
            logits = self.classifier(sequence_output_1)
            loss = None
            if labels_1 is not None:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))
         
        # Return logits, loss, and hidden states
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## MixupDataset

In [7]:
def get_mixup_data(data):
    
    # Remove none and hard examples
    data = data[(data['category'] != 'none') & (data['category'] != 'hard')].reset_index(drop=True)
    
    # Same class mixup    
    data_easy = data[data['category'] == 'easy'].reset_index(drop=True)
    temp_easy = data_easy[['idx', 'label', 'category']].copy().rename(columns={"idx": "idx_2", "label": "label_2", "category": "category_2"}).sample(frac=1).reset_index(drop=True)
    data_easy = pd.concat([data_easy, temp_easy], axis=1)

    data_ambiguous = data[data['category'] == 'ambiguous'].reset_index(drop=True)
    temp_ambiguous = data_ambiguous[['idx', 'label', 'category']].copy().rename(columns={"idx": "idx_2", "label": "label_2", "category": "category_2"}).sample(frac=1).reset_index(drop=True)
    data_ambiguous = pd.concat([data_ambiguous, temp_ambiguous], axis=1)
        
    same_data = pd.concat([data_easy, data_ambiguous]).sample(frac=1).reset_index(drop=True)
    same_data['mixup_type'] = 'same'
    
    # Different class mixup
    data_easy = data[data['category'] == 'easy'].reset_index(drop=True)
    data_ambiguous = data[data['category'] == 'ambiguous'].reset_index(drop=True)

    easy_tuple = list(zip(data_easy['idx'].tolist(), data_easy['label'].tolist(), data_easy['category'].tolist()))
    ambiguous_tuple = list(zip(data_ambiguous['idx'].tolist(), data_ambiguous['label'].tolist(), data_ambiguous['category'].tolist()))

    ambiguous4easy = random.choices(ambiguous_tuple, weights=np.ones(len(ambiguous_tuple)), k=len(data_easy))
    easy4ambiguous = random.choices(easy_tuple, weights=np.ones(len(easy_tuple)), k=len(data_ambiguous))

    ambiguous4easy = pd.DataFrame(ambiguous4easy, columns=['idx_2', 'label_2', 'category_2'])
    data_easy = pd.concat([data_easy, ambiguous4easy], axis=1)

    easy4ambiguous = pd.DataFrame(easy4ambiguous, columns=['idx_2', 'label_2', 'category_2'])
    data_ambiguous = pd.concat([data_ambiguous, easy4ambiguous], axis=1)

    different_data = pd.concat([data_easy, data_ambiguous]).sample(frac=1).reset_index(drop=True)
    different_data['mixup_type'] = 'different'
    
    # Non-mixup data
    data['idx_2'] = None
    data['label_2'] = None
    data['category_2'] = None
    data['mixup_type'] = 'non_mixup'
    
    return pd.concat([data, same_data, different_data]).sample(frac=1).reset_index(drop=True)

In [9]:
class MixupDataset(Dataset):
    
    def __init__(
        self, 
        tokenizer, 
        data: pd.DataFrame,
        sampling_type: str
    ):
        self.data = data
        self.sampling_type = sampling_type
        
        if self.sampling_type == 'sequential':
            sorting_dict = {
                'non_mixup_easy': 0,
                'non_mixup_ambiguous': 1,
                'same_easy': 2,
                'different_easy': 3,
                'same_ambiguous': 4,
                'different_ambiguous': 5
            }
            self.data['data_type'] = self.data['mixup_type'] + '_' + self.data['category']
            self.data = self.data.iloc[self.data.data_type.map(sorting_dict).argsort()].reset_index(drop=True)
            
        self.tokenizer = tokenizer
        self.tokenized_data = tokenizer.batch_encode_plus(
            self.data[INPUT_COLUMN].tolist(),
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_token_type_ids=True,            
            return_tensors='pt'
        )
        

                
    def __len__(
        self
    ):
        return len(self.data)
    
    
    
    def __getitem__(
        self,
        index: int
    ):
        data = {
            'data_type': self.data_type,
            'input_ids_1': self.tokenized_data['input_ids'][index].flatten(),
            'attention_mask_1': self.tokenized_data['attention_mask'][index].flatten(),
            'labels_1': torch.tensor(self.data.iloc[index][OUTPUT_COLUMN], dtype=torch.long),
        }
        
        if idx_2 is not None:
            idx2 = self.data.iloc[index]['idx_2']
            index2 = self.data[self.data['idx'] == idx2].index[0]

            data['input_ids_2'] = self.tokenized_data['input_ids'][index2].flatten()
            data['attention_mask_2'] = self.tokenized_data['attention_mask'][index2].flatten()
            data['labels_2'] = torch.tensor(self.data.iloc[index2][OUTPUT_COLUMN], dtype=torch.long)
            
        return data

## Training loop

In [ ]:
def train(model, tokenizer, optimizer, criterion, device, train_loader, num_epochs, output_dir):
    losses = []
    train_iterator = trange(int(num_epochs), desc='Epoch')
    for _ in train_iterator:
        tr_loss = 0
        step = None
        epoch_iterator = tqdm(train_loader, desc='Train Iteration')
        for step, batch in enumerate(epoch_iterator):
            model.train()
            inputs = {k:v.to(device) for k, v in batch.items()}
            labels = inputs['labels_1']

            optimizer.zero_grad()

            out = model(**inputs)[1].double().to(device)

            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

            tr_loss += loss.item()
        losses.append(tr_loss/(step+1))
        print('train loss: {}'.format(tr_loss/(step+1)))

    # save model and tokenizer
    print('Saving model and tokenizer')

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [ ]:
def eval(model, eval_loader, device, criterion=nn.CrossEntropyLoss(), with_labels=True):
    probs = None
    gold_labels = None

    eval_loss = 0
    step = None
    eval_iterator = tqdm(eval_loader, desc='Evaluating')
    for step, batch in enumerate(eval_iterator):
        model.eval()

        with torch.no_grad():
            
            inputs = {k:v.to(device) for k, v in batch.items()}
            labels = inputs['labels_1']
            del inputs['labels_1']
            del inputs['labels_2']
 
            out = model(**inputs)[0].double()
            out = F.softmax(out, dim=1)

            if with_labels:
                labels = batch[2].to(device)
                loss = criterion(out, labels)

            if probs is None:
                probs = out.detach().cpu().numpy()
                if with_labels:
                    gold_labels = labels.detach().cpu().numpy()
            else:
                probs = np.append(probs, out.detach().cpu().numpy(), axis=0)
                if with_labels:
                    gold_labels = np.append(gold_labels, labels.detach().cpu().numpy(), axis=0)

            if with_labels:
                eval_loss += loss.item()
    
    if with_labels:
        eval_loss /= (step+1)
        print('eval loss: {}'.format(eval_loss))

        # compute accuracy
        preds = np.argmax(probs, axis=1)
        accuracy = np.sum(preds == gold_labels)/len(preds)
        print('eval accuracy: {}'.format(accuracy))

    return probs

In [ ]:
def main():
    # create argument parser
    parser = argparse.ArgumentParser()

    parser.add_argument('--task_name', help='Task to fine-tune RoBERTa on', default='sst2')
    parser.add_argument('--roberta_version', type=str, default='roberta-base', help='Version of RoBERTa to use')
    parser.add_argument('--device', type=int, default=0, help='which GPU to use')
    parser.add_argument('--sampling_type', type=str, default='random', help='How to sample data')
    parser.add_argument('--num_epochs', type=int, default=3, help='Number of epochs to fine-tune')
    parser.add_argument('--max_seq_length', type=int, default=None, help='Maximum sequence length of the inputs')
    parser.add_argument('--batch_size', type=int, default=16, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=1e-5, help='Adam learning rate')
    parser.add_argument('--output_dir', type=str, default='roberta_mixup_ckpts/', help='Directory to save fine-tuned models')
    parser.add_argument('--seed', type=int, default=42, help='Random seed for initialization')
    parser.add_argument('--file_format', type=str, default='.tsv', help='Data file format for tasks not available for download at HuggingFace Datasets')
    parser.add_argument('--train_file', type=str, default=None, help='LM txt file')
    parser.add_argument('--val_file', type=str, default=None, help='LM txt file')
    parser.add_argument('--num_labels', type=int, default=2, help='Number of labels in training data')
    parser.add_argument('--fname', type=str, default=None, help='MSP output file')
    parser.add_argument('--n', type=int, default=None, help='Number of examples to process (for debugging)')
device
    args = parser.parse_args()

    # set device
    device = torch.device(f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu')

    num_labels = args.num_labels
    
    # set seed
    set_seed(args.seed)

    # load RoBERTa tokenizer and model
    print('Loading RoBERTa tokenizer and model')

    tokenizer = RobertaTokenizer.from_pretrained(args.roberta_version, cache_dir='cache/huggingface/transformers')
    model = RobertaMixerForSequenceClassification.from_pretrained(args.roberta_version, cache_dir='cache/huggingface/transformers').to(device)
    model.resize_token_embeddings(len(tokenizer))

    # process dataset
    print('Processing dataset')

    # Process train dataset
    train_df = pd.read_csv(args.train_file)
    train_dataset = MixupDataset(tokenizer=tokenizer, data=train_df, sampling_type=args.sampling_type)
    train_loader = Dataloader(train_dataset)
    
    # Process eval dataset
    eval_df = pd.read_csv(args.val_file)
    eval_dataset = MixupDataset(tokenizer=tokenizer, data=eval_df, sampling_type=args.sampling_type)
    eval_loader = Dataloader(eval_dataset)

    # instantiate optimizer and criterion
    optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)
    criterion = nn.CrossEntropyLoss()

    # fine-tune model 
    if train_loader is not None:
        print('Fine-tuning model')
        train(model, tokenizer, optimizer, criterion, device, train_loader, args.num_epochs, args.output_dir)

    # evaluate model
    if eval_loader is not None:
        print('Evaluating model')
        probs = eval(model, eval_loader, device, criterion, with_labels=with_labels)
        np.save(os.path.join(SAVE_PATH, f'{args.fname}_probs'), probs)
        msp = np.max(probs, axis=1)
        if args.fname is not None:
            np.save(os.path.join(SAVE_PATH, f'{args.fname}_msp'), msp)